## Estimating rotors using dual quaternions 

This is a comparison between the method developed in pygacal and the method presented by Bayro-Corrochano et.al. in [Motor Algebra for 3D Kinematics: The Case of the Hand-Eye Calibration](https://link.springer.com/article/10.1023%2FA%3A102656781298). 

Representing lines as 

$$ L = n + I m $$

$$ n = x_2 - x_1 $$

$$ m = x_1 \times x_2 $$

$$ \
\begin{bmatrix} 
    a - b & [a + b]_{\times} & 0_{3 \times 1} & 0_{3 \times 1} \\
    a' - b' & [a' + b']_{\times} & a - b & [a + b]_{\times} \\
\end{bmatrix} \begin{bmatrix} 
    R \\
    R'
\end{bmatrix} = \begin{bmatrix} 
    0 \\ 
    0
\end{bmatrix} $$

$$
    \Rightarrow \begin{bmatrix} 
    C_1 \\
    \vdots \\
    C_N
\end{bmatrix} \begin{bmatrix} 
    R \\
    R'
\end{bmatrix} = \begin{bmatrix} 
    0 \\ 
    0
\end{bmatrix}
$$

$$ \Rightarrow D \begin{bmatrix} 
    R \\
    R'
\end{bmatrix} = \begin{bmatrix} 
    0 \\ 
    0
\end{bmatrix}$$

$$ D = U \Sigma V^T $$

$\Sigma$ should have two and only two small singular values. 

Let $v_7 = \begin{bmatrix} u_1 \\ v_1 \end{bmatrix} $ and $v_8 = \begin{bmatrix} u_2 \\ v_2 \end{bmatrix} $ 

Solve the set of equations for $ \alpha \neq 0 $ and $ \beta \neq 0$: 

\begin{align}
    \alpha^2 u_1^T u_1 + 2 \alpha \beta  u_1^T u_2 + \beta^2  u_2^T u_2 &= 1 \\
    \alpha^2 u_1^T v_1 + \alpha \beta ( u_1^T v_2 + v_1^T u_2) + \beta^2  u_2^T u_2 &= 1 
\end{align}

Choose largest $\beta$ as our solution. Sign does not matter. 

$$ \Rightarrow \begin{bmatrix} 
    R \\
    R'
\end{bmatrix} = \alpha v_7 + \beta v_8 $$

$R$ is a quaternion representing the rotation
$R'$ can be converted to $t$

$$ \begin{bmatrix} 0 \\t \end{bmatrix}  = - 2 R' \tilde{R} $$ (quaternion multiplication)


In [1]:
from pygacal.common.cgatools import *
from pygacal.geometry import *
from pygacal.geometry.lines import *
from pygacal.geometry.transformations import * 
from pygacal.rotation.mapping import BivectorLineMapping
from pygacal.rotation import minimizeError
from pygacal.rotation.costfunction import rotorAbsCostFunction


import numpy as np


In [2]:


def cross_product(a):
    return np.array([[0, -a[2], a[1]], [a[2], 0, -a[0]], [-a[1], a[0], 0]])

def dual_quat_line(x_1, x_2):
    n = x_2 - x_1
    m = np.cross(x_1, x_2)
    return n, m

def D_matrix(a, ad, b, bd):
    D = np.zeros((6, 8))
    D[:3, 0]   = a - b
    D[:3, 1:4] = cross_product(a + b)
    D[3:, 0]   = ad - bd
    D[3:, 1:4] = cross_product(ad + bd)
    D[3:, 4]   = a - b
    D[3:, 5:] = cross_product(a + b)
    return D


    

In [3]:
seed = 21
sigma_T = 0.4
sigma_R = 0.05
N = 200

line1, line2 = createRandomLines(2)
R_real = RotorLine2Line(line1, line2)

traininglinesets = createNoisyLineSet(R_real, sigma_R, sigma_T, N)

In [4]:
C = np.zeros((N*6, 8))

for i in range(len(traininglinesets)):
    linepair = traininglinesets[i]
    L1 = linepair[0]
    L2 = linepair[1]    
    a, ma = findLineParams(L1)
    b, mb = findLineParams(L2)
        
    x11 = MVto3DVec(a)
    x12 = MVto3DVec(a) + MVto3DVec(ma)
    x21 = MVto3DVec(b)
    x22 = MVto3DVec(b) + MVto3DVec(mb)
    
    a, ad = dual_quat_line(x11, x12)
    b, bd = dual_quat_line(x21, x22)   
    
    C[6*i:6*(i + 1), :] = D_matrix(a, ad, b, bd)

In [5]:
U, S, VH = np.linalg.svd(C)

In [6]:
V = VH.transpose()

v7 = V[:, 6]
v8 = V[:, 7]

u1 = v7[:4]
v1 = v7[4:]

u2 = v8[:4]
v2 = v8[4:]

a = np.dot(u1, v1)
b = np.dot(u1, v2) + np.dot(v1, u2)
c = np.dot(u2, v2)

mu_1 = (-b + np.sqrt(b*b - 4 * a * c))/(2 * a)
mu_2 = (-b - np.sqrt(b*b - 4 * a * c))/(2 * a)

a = np.dot(u1, u1)
b = 2 * np.dot(u1, u2) 
c = np.dot(u2, u2)

beta_1 = 1.0 / np.sqrt(mu_1*mu_1 * a + mu_2 * b + c)
beta_2 = 1.0 / np.sqrt(mu_2*mu_2 * a + mu_2 * b + c)

beta = min(beta_1, beta_2)
mu = ([mu_1, mu_2])[np.argmin([[beta_1, beta_2]])]


alpha = mu * beta
r = alpha * v7 + beta * v8

print(alpha, beta)

-1.217867543666495 0.0015822324277964773


In [7]:
r

array([ 0.66324755,  0.5126254 , -0.41023037, -0.35923151,  0.16012357,
       -0.38960053, -0.47534342,  0.28249957])

In [8]:
quat = r[:4]
#quat = r[:4]/np.sqrt(np.dot(r[:4], r[:4]))
#print(quat)
angles = quaternion_to_rotation(quat) 
print(angles)

[-1.22713241  1.14928337  0.1371147 ]


In [9]:
#dual_quat = r[4:]/np.sqrt(np.dot(r[4:], r[4:]))
#print(dual_quat)
dual_quat = r[4:]

def quaternion_mult(q,r):
    w0, x0, y0, z0 = q
    w1, x1, y1, z1 = r
    return np.array([-x1*x0 - y1*y0 - z1*z0 + w1*w0,
                         x1*w0 + y1*z0 - z1*y0 + w1*x0,
                        -x1*z0 + y1*w0 + z1*x0 + w1*y0,
                         x1*y0 - y1*x0 + z1*w0 + w1*z0], dtype=np.float64)

def quat_conjugate(quat):
    return np.array([quat[0],-1*quat[1],-1*quat[2],-1*quat[3]])

t = -2 * quaternion_mult(dual_quat, quat_conjugate(quat))

translation = t[1:]
print(translation)


[ 0.10767353  0.48944628 -1.29677532]


In [10]:
def bivector_difference(R_true, R_test):
    B_true = ga_log(R_true)
    B_test = ga_log(R_test) 
    return rotorAbsCostFunction(ga_exp(B_test - B_true))
    

In [11]:
R_min, nit = minimizeError(traininglinesets, mapping = BivectorLineMapping, x0 = None)   


In [12]:
print(versor_to_param(R_real))
print(bivector_difference(R_real, R_real))

(array([-1.16366943,  1.062581  ,  0.19182439]), array([-0.00149985,  0.31151166, -0.63195071]))
0.0


In [13]:
R_Eduardo = parameters_to_versor(angles, translation)
print(versor_to_param(R_Eduardo))
print(bivector_difference(R_real, R_Eduardo))

(array([-1.22713241,  1.14928337,  0.1371147 ]), array([ 0.10767353,  0.48944628, -1.29677532]))
0.1561981664124321


In [14]:
print(versor_to_param(R_min))
print(bivector_difference(R_real, R_min))

(array([-1.144068  ,  1.08023709,  0.17233148]), array([ 0.10039105,  0.24129895, -0.76492784]))
0.009811315567573912
